# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

    # Uncomment next line if you want to check the paths retrieved    
    #print(file_path_list)

/Users/marcelovolta/repos/udacity_data_eng_nd/project_02/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables. 
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'], port=6000) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




We will create the tables in Cassandra according to the following table that lists the requirements for each query: 

| Query | Filter_01 | Filter_02 | Sort | Field_01 | Field_02 | Field_03 |    
| --- | --- | --- |
| 1 | sessionid | iteminsession | None | artist | song | length |
| 2 | userid | sessionid | iteminsession | artist | song | firstname + lastname |
| 3 | song | None | None | firstname + lastname | None | None |

The approach followed is to use the column(s) included in the filter definition for the partition key, and the column(s) used for sorting, in the clustering key. 

In [8]:
#Query_01 - Table creation: events_session_item

drop_query = 'DROP TABLE IF EXISTS events_session_item'
create_query = 'CREATE TABLE IF NOT EXISTS events_session_item '
create_query = create_query + '(sessionid int, iteminsession int, artist text, song text, length decimal, PRIMARY KEY (sessionid, iteminsession))'

try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)
    


In [11]:
%%time

# events_session_item: Insertion of records

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO events_session_item (sessionid, iteminsession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

CPU times: user 4.9 s, sys: 1.26 s, total: 6.16 s
Wall time: 25.8 s


#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# run Query_01 
# The events_session_item table has a partition key composed of the sessionid and iteminsession columns as these
# are included in the filter. These two are enough to uniquely identify a row and also there is no sorting needed
# in the query (we will typically obtain a single row), so clustering key was not defined

select_query = 'SELECT artist, song, length FROM events_session_item '
select_query = select_query + 'WHERE sessionId = 338 AND itemInSession = 4'
rows = session.execute(select_query)

for row in rows:
    print ('Artist: {0} - Song: {1} - Length: {2}'.format(row.artist, row.song, row.length))

Artist: Faithless - Song: Music Matters (Mark Knight Dub) - Length: 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
#Query_02 - Table creation: events_user_session

drop_query_2 = 'DROP TABLE IF EXISTS events_user_session'
create_query = 'CREATE TABLE IF NOT EXISTS events_user_session '
create_query = create_query + '(userid int, sessionid int, iteminsession int, artist text, song text, name text, PRIMARY KEY ((userid, sessionid), iteminsession))'

try:
    session.execute(drop_query_2)
    session.execute(create_query)
except Exception as e:
    print(e)


In [17]:
%%time

# events_user_session: Insertion of records

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO events_user_session (userId, sessionId, itemInSession, artist, song, name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], (line[1] + ' ' + line[4])))

CPU times: user 4.82 s, sys: 1.25 s, total: 6.07 s
Wall time: 23.4 s


In [18]:
#run Query_02
# For the events_user_session, the partition key is composed by userid and sessionid as these are included in the
# where clause. Also, since during a session a user can listen to more than one song we need to add another column 
# to the primary key to make it unique. Since we also need to sort the results by iteminsession, that column is
# the natural choice for a clustering key

select_query = 'SELECT artist, song, name FROM events_user_session '
select_query = select_query + 'WHERE userId = 10 AND sessionId = 182'
rows = session.execute(select_query)

for row in rows:
    print ('Artist: {0} - Song: {1} - User: {2}'.format(row.artist, row.song, row.name))

Artist: Down To The Bone - Song: Keep On Keepin' On - User: Sylvie Cruz
Artist: Three Drives - Song: Greece 2000 - User: Sylvie Cruz
Artist: Sebastien Tellier - Song: Kilometer - User: Sylvie Cruz
Artist: Lonnie Gordon - Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - User: Sylvie Cruz


In [10]:
# Query_03 table creation: events_song

drop_query_3 = 'DROP TABLE IF EXISTS events_song'
create_query = 'CREATE TABLE IF NOT EXISTS events_song '
create_query = create_query + '(song text, userid int, name text, PRIMARY KEY (song, userid))'

try:
    session.execute(drop_query_3)
    session.execute(create_query)
except Exception as e:
    print(e)


In [15]:
%%time

#events_song: insertion of records

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO events_song (song, userid, name) "
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), (line[1] + ' ' + line[4])))
        
        

CPU times: user 5.34 s, sys: 1.27 s, total: 6.61 s
Wall time: 28.2 s


In [16]:
#run Query_03
# The events_song table will be queried with a song name in the where clause. To make the rows unique we have added 
# the userid as a clustering key, since we only need each song-user combination to appear once

select_query = 'SELECT name FROM events_song '
select_query = select_query + "WHERE song = 'All Hands Against His Own'"
rows = session.execute(select_query)

for row in rows:
    print (row.name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [22]:
#Prepare the table
drop_query = 'DROP TABLE IF EXISTS events_session_item'
create_query = 'CREATE TABLE IF NOT EXISTS events_session_item '
create_query = create_query + '(sessionid int, iteminsession int, artist text, song text, length decimal, PRIMARY KEY (sessionid, iteminsession))'

try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)


In [23]:
%%time 

#first method: Using direct inserts and Python File I/O capabilities

#Extraction and Load
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO events_session_item (sessionid, iteminsession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

        

CPU times: user 5.08 s, sys: 1.27 s, total: 6.35 s
Wall time: 26.9 s


## Additional Tests

To try to improve the loading time, I considered using the BATCH statement in Cassandra, but found out that this is an antipattern if used for rows that will be recorded in more than one node and it is only recommended when the records to be inserted have the same value of Primary Key. 
I tried the Pandas approach both to open the file and use it in the insertion, plus the PreparedStatems feature (method #02 and finally a combination of the first method (file I/O and PreparedStatements), which I called method #03. 
To compare the efficiency of each method, I used the %%time magic. 

Results are shown below. The best method is the first one, the original approach used in the demos of the lesson.

In [25]:
#Prepare the table
drop_query = 'DROP TABLE IF EXISTS events_session_item'
create_query = 'CREATE TABLE IF NOT EXISTS events_session_item '
create_query = create_query + '(sessionid int, iteminsession int, artist text, song text, length decimal, PRIMARY KEY (sessionid, iteminsession))'

try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)
    

In [27]:
%%time
#Method 02: Use Pandas and prepared Statements
#(Reference: https://medium.com/swlh/building-a-python-data-pipeline-to-apache-cassandra-on-a-docker-container-fc757fbfafdd)

#Open file in Pandas
df = pd.read_csv(file)

#Prepare the CQL statement
query_insert = "INSERT INTO events_session_item " 
query_insert = query_insert + "(sessionid, iteminsession, artist, song, length) "
query_insert = query_insert + "VALUES (?, ?, ?, ?, ?)"

prepared = session.prepare(query_insert)

#Run Insertion
for index, row in df.iterrows():
    session.execute(prepared
                    , (row['sessionId']
                    , row['itemInSession']
                    , row['artist']
                    , row['song']
                    , row['length']))



CPU times: user 9.01 s, sys: 1.36 s, total: 10.4 s
Wall time: 31.8 s


In [28]:
#Prepare the table
drop_query = 'DROP TABLE IF EXISTS events_session_item'
create_query = 'CREATE TABLE IF NOT EXISTS events_session_item '
create_query = create_query + '(sessionid int, iteminsession int, artist text, song text, length decimal, PRIMARY KEY (sessionid, iteminsession))'

try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)

In [29]:
%%time
#Method 03: No Pandas, prepared statement

prepared = session.prepare(query_insert)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(prepared, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        

CPU times: user 5.65 s, sys: 1.26 s, total: 6.91 s
Wall time: 24.2 s


### Drop the tables before closing out the sessions

In [30]:
session.execute(drop_query)
session.execute(drop_query_2)
session.execute(drop_query_3)

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()